<a href="https://colab.research.google.com/github/devorahst/Test/blob/main/DVariable.CODISNDISeligibleProfile/Section_2_StrangulationAndAssultiveActs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-Up

In [ ]:
#pulls up our SAK dataset
#@title Upload Dataset
file_id = "13DLmmbYXonl9alHR4VobfeTuA_IxlQxZ" #@param {type:"string"}
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  _, done = downloader.next_chunk()

fileId = drive.CreateFile({'id': file_id }) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])  
fileId.GetContentFile(fileId['title'])  # Save Drive file as a local file

!pip install -U scikit-learn

from scipy.stats import chi2_contingency
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

with open(fileId['title'], encoding="utf8", errors='ignore') as f:
  df = pd.read_csv(f)

df = df.apply(pd.to_numeric, errors='ignore')
predictedVariable = "CODISNDISeligibleProfile"


#delete rows that don't have a value for CODISNDISeligibleProfile

predictedFeatures = ['CODISNDISeligibleProfile', 'SDISeligibleprofile']  

numericalFeatures = ['Age', 'Timebetweenassaultandexaminhours', 'PainLevel', 'MulitipleSuspectNumber', 
                     'NumberofUnknownresponses', 'NumberAssaultiveActs', 'Numberofphysicalinjuries', 'Numberofgentialinjuries',
                     'NumberOFitemsTested', 'TimeBetweenCollectAndDNAext', 'TimeBetweenSubmissionANDtesting', 'NumberOfswabsQuantMaleDNA',
                     'NumberOfswabsDNAanalysis', 'NumberofSTRDNAloci', 'NumberOFswabsSTRDNAprofile', 'NumberOfYSTRDNAloci']

categoricalFeatures = ['Site', 'EXAMbySANE', 'YearKitCollected', 'KITbroughtTOcrimelab', 'KITlengthofSubmissionTime',
                       'UnderAge18', 'Gender', 'ExamDeclined', 'Noninterview', 'Race', 'PriorHxofSAover14',
                       'PriorHxofSAunder14', 'Student', 'Military', 'Pain', 'PainLocation1','PainLocation2', 
                       'PainLocation3', 'PainLocation4','PainTreatment', 'PermanentAddress', 'CurrentPhysicalmedprob',
                       'MedProbChronic', 'MedProbInfection', 'MedProbBlood', 'MedProbCardiac', 'MedProbEar', 'MedProbEndocrine',
                       'MedProbEye', 'MedProbGI', 'MedProbGU', 'MedProbGYN', 'MedProbImmune', 'MedProbMusculoskeletal', 'MedProbNeurological',
                       'MedProbOral', 'MedProbRenal', 'MedProbRespiratory', 'MedProbSkin', 'MedProbOther', 'Medication',
                       'PsychotropicMEDuse', 'PsychotropicANTIPSYCHOTICSatypical', 'PsychotropicSTIMULANTuse', 'PsychotropicANTIANXIETY', 
                       'PsychotropicANTIDEPRESSANTS', 'PsychotropicANTISEIZUREbipolar', 'PsychotropicADDICTIONmeds','PsychotropicSLEEPaid', 'PsychotropicOTHER', 
                       'PsychotropicANTIPSYCHOTICStypical', 'PolypharmacyPsychMeds', 'ImmunizationstatusTETANUS', 'ReceivedTetanus',
                       'ImmunizationstatusHEP', 'ReceivedHepB', 'Sexualcontactwithin120hours', 'Selfdisclosurementalillness', 'MIdepression',
                       'MIanxiety', 'MIPTSD', 'MIpsychoticDisorders', 'MIadhd', 'MIpersonalitydisorder', 'MIbipolar', 'MIeatingdisorder', 'MIdrugalcoholdisorders', 
                       'MIother', 'SelfDiscolsureMentalillnessORuseofpsychotropics', 'OnlineMeetingOFsuspect', 'Suspectrelationship',
                       'Locationofassault', 'PatientActionScratch', 'PatientActionBite', 'PatientActionHit', 'PatientActionKick', 'PatientActionOther',
                       'Suspectrace', 'SuspectactionVERBAL', 'SuspectactionsGRABBEDHELD', 'SuspectactionsPHYSICALBLOWS', 'SuspectactionsSTRANGLEDCHOKED',
                       'SuspectactionsWEAPON', 'SuspectactionsRESTRAINTS', 'SuspectactionsBURNED', 'MultipleSuspects', 'SuspectedDrugfacilitated',
                       'Patientdruguse', 'PatientETOHuse', 'Suspectdruguse', 'SuspectETOHuse', 'PatientSuspectETOHordrug', 'LossOFconsciousnessORawareness',
                       'OneORmoreunknownanswer', 'Unknownanswerto4ormorequestions', 'UnknownanswertoALL', 'AsleepANDawakenedtoassault', 'MemoryLoss',
                       'LossOfconsciousness', 'DecreasedAwareness', 'TonicImmobility', 'Detachment', 'NOSApatientsVAGINApenis', 'NOSApatientsVAGINAfingerhand',
                       'NOSApatientsVAGINAmouth', 'NOSApatientsVAGINAobject', 'NOSApatientsANUSpenis', 'NOSApatientsANUSfingerhand', 'NOSApatientsANUSmouth', 
                       'NOSApatientsANUSobject', 'NOSApatientsPENISgenitals', 'NOSApatientsPENISfinger', 'NOSApatientsPENISmouth', 'NOSApatientsPENISobject', 
                       'NOSApatientsMOUTHpenis', 'NOSApatientsMOUTHfinger', 'NOSApatientsMOUTHmouth', 'NOSApatientsMOUTHobject', 'SUSPECTmouthcontactGENITALS', 
                       'SUSPECTmouthcontactMOUTH', 'SUSPECTmouthcontactOTHER', 'HANDSofSuspectBreast', 'HANDSofSuspectExtremities', 
                       'HANDSofSuspectOther', 'Ejaculation', 'CONDOMuse', 'LUBRICATIONuse', 'SuspectWASHEDpatient', 'SuspectINJUREDbypatient', 'PostassaultURINATED', 
                       'PostassaultDEFECATED', 'PostassaultDOUCHED', 'PostassaultVOMITED', 'PostassaultGARGLED', 'PostassaultBRUSHEDTEETH', 'PostassaultATEdrank', 
                       'PostassaultBATHED', 'PostassaultGENITALWIPE', 'PostassaultCHANGEDCLOTHING', 'PostassaultREMOVEDInserted', 'PhysicalORmentalimpairment', 'Physicalinjury', 
                       'LPIhead', 'LPIneck', 'LPIbreasts', 'LPIchestback', 'LPIabdomen', 'LPIextremities', 'TPIlaceration', 'TPIecchymosis', 'TPIabrasion', 'TPIredness', 
                       'TPIswelling', 'TPIbruise', 'TPIpetechiae', 'TPIincision', 'TPIavulsion', 'TPIdiscoloredmark', 'TPIpuncturewound', 'TPIfracture', 
                       'TPIbitemark', 'TPIburn', 'TPImissingorbrokenTEETH', 'TPIconjunctivalhemorrhage', 'Genitalinjury', 'LGIinnerthighs', 'LGIclitoralhoodclitoris', 
                       'LGIlabiamajora', 'LGIlabiaminora', 'LGIperiurethraltissueURETHRA', 'LGIperihymenaltissue', 'LGIhymen', 'LGIvagina', 'LGIcervix', 'LGIfossanavicularis', 
                       'LGIposteriorfourchette', 'LGIperineum', 'LGIperineum', 'LGIanalrectal', 'LGIbuttocks', 'LGImalePerianalperineum', 'LGIglanspenis', 'LGIpenileshaft', 
                       'LGImaleURETHRALmeatus', 'LGIscrotum', 'LGItestes', 'LGImaleanus', 'LGImalerectum', 'TGIlaceration', 'TGIecchymosis', 'TGIabrasion', 'TGIredness', 
                       'TGIswelling', 'TGIbruise', 'TGIpetechiae', 'TGIincision', 'TGIavulsion', 'TGIdiscoloredmark', 'TGIpuncturewound', 'ToludineDYEuptake', 'HIVnPEP', 
                       'UQuikcollected', 'Yscreen', 'NumberItemsWITH3cutoff', 'ItemsAnalyzed1', 'ItemsAnalyzed2', 'ItemsAnalyzed3', 'ItemsAnalyzed4', 'ItemsAnalyzed5', 
                       'ItemsAnalyzed6', 'ItemsAnalyzed7', 'ItemsAnalyzed8', 'ItemsAnalyzed9', 'ItemsAnalyzed10', 'TypesOFitemsTested', 'RandomSample20142015', 
                       'YearofDNAextraction', 'LocationOfTesting','DANYfundedSAK', 'DNAKitUsed', 'SerologyDoneBeforeDNA', 'QuantMaleDNAFound', 'QuantMaleSwabLoc1', 
                       'QuantMaleSwabLoc2', 'QuantMaleSwabLoc3', 'QuantMaleSwabLoc4', 'QuantMaleSwabLoc5', 'ProbableSTRDNAprofileOFsuspect', 'ProfileofSTRDNAloci', 'ProbableYSTRDNAprofile', 'ProfileOfYSTRDNAloci', 
                       'SwabLocationYSTRDNA', 'SecondSwabLocationYSTRDNA', 'SwabFromSuspectwithVictimDNA', 'ExcludeSuspect', 'ConsensualPartnerStandardSubmitted', 
                       'STRDNAProbableprofileTYPE', 'CODISprofileHit', 'STRDNAkitUsed', 'SUSPECTmouthcontactBREASTS', 'Swab1LocationSTRDNAprofile', 'Swab2LocationSTRDNAprofile',
                       'Swab3LocationSTRDNAprofile', 'SuspectStandardSubmitted', 'CODISNDISreasons', 'CODISSDISreasons']

#unusedFeatures and stringFeatures are columns that contain data that was relevant to medical professionals and for legal purposes, 
#but that aren't useful for our feature association or for predicting eligibility
unusedFeatures = ['filter_$', 'PainTreatmentYesNo', 'GenderMaleFemale', 'DVsuspect', 'RacePrimaryGroups', 'IPSAcombined', 'STRDNAcompleted', 
                  'PhysicalInjuryNOunknown', 'GenitalInjuryNOunknown']

stringFeatures = ['DeIdentifiedCase', 'Raceother', 'SchoolName', 'MilitaryBranchName', 'AddressIfnotPermanent', 'Currentmedprobtext',
                  'MedProbOtherText', 'Medicationtext', 'Sexualcontactwithin120hoursTYPE', 'SelfdisclosureMItype', 'OnlineMeetingName', 'SuspectrelationshipOTHER',
                  'LocationofassaultOTHER', 'Surfaceofassault', 'PatientActionOtherTEXT', 'SuspectraceOTHER', 'SuspectOTHERactions', 'NOSApatientsVAGINAobjectdescription',
                  'NOSApatientsANUSobjectdescription', 'NOSApatientsPENISobjectdescription', 'NOSApatientsMOUTHobjectdescription', 'EjaculationSITE', 'LUBRICATIONtype',
                  'SuspectINJUREDbypatientexplanation', 'Impairmentdescription', 'UBFSnumber', 'ISPnumber', 'DateSubmittedUBFS', 'DateofDNAextractionReport',
                  'BodySwabLocQuant', 'BodySwabDNAanalysis', 'BodySwabLocationSTRDNA', 'BodySwabYSTRDNA', 'ISPnotes2020', 'UBFSnotes2020', 'UBFSnotes2018', 'SUSPECTmouthcontactOTHERsite', 'UBFSnotes2014']

dummy1 = pd.get_dummies(df['Swab1ToDNAanalysis'])
dummy2 = pd.get_dummies(df['Swab2ToDNAanalysis'])
dummy3 = pd.get_dummies(df['Swab3ToDNAanalysis'])
dummy4 = pd.get_dummies(df['Swab4ToDNAanalysis'])

dummy4['0'] = dummy1['0']
dummy1['11'] = dummy2['11']
dummy3['0'] = dummy1['0']

newDummy = dummy1


newDummy = newDummy.where(newDummy != 0, dummy2)
newDummy = newDummy.where(newDummy != 0, dummy3)
newDummy = newDummy.where(newDummy != 0, dummy4)

# print(newDummy)
df['SwabToDNAanalysisNoquantmaleDNAfound'] = newDummy['0'].astype('category')
df['SwabToDNAanalysisVaginal'] = newDummy['1']
df['SwabToDNAanalysisCervical'] = newDummy['2']
df['SwabToDNAanalysisPerianal'] = newDummy['3']
df['SwabToDNAanalysisRectal'] = newDummy['4']
df['SwabToDNAanalysisOral'] = newDummy['5']
df['SwabToDNAanalysisBody'] = newDummy['6']
df['SwabToDNAanalysisUnderwear'] = newDummy['7']
df['SwabToDNAanalysisOtherClothing'] = newDummy['8']
df['SwabToDNAanalysisBedding'] = newDummy['9']
df['SwabToDNAanalysisCondom'] = newDummy['10']
df['SwabToDNAanalysisTampon'] = newDummy['11']

swabToDNAFeatures = ['SwabToDNAanalysisNoquantmaleDNAfound', 'SwabToDNAanalysisVaginal', 'SwabToDNAanalysisCervical', 'SwabToDNAanalysisPerianal', 'SwabToDNAanalysisRectal', 
                     'SwabToDNAanalysisOral','SwabToDNAanalysisBody', 'SwabToDNAanalysisUnderwear', 'SwabToDNAanalysisOtherClothing', 'SwabToDNAanalysisBedding', 
                     'SwabToDNAanalysisCondom','SwabToDNAanalysisTampon' ]

categoricalFeatures.extend(['SwabToDNAanalysisNoquantmaleDNAfound', 'SwabToDNAanalysisVaginal', 'SwabToDNAanalysisCervical', 'SwabToDNAanalysisPerianal', 'SwabToDNAanalysisRectal', 
                     'SwabToDNAanalysisOral','SwabToDNAanalysisBody', 'SwabToDNAanalysisUnderwear', 'SwabToDNAanalysisOtherClothing', 'SwabToDNAanalysisBedding', 
                     'SwabToDNAanalysisCondom','SwabToDNAanalysisTampon'])

df = df.replace(r'^\s+$', np.nan, regex=True)
df = df.replace({np.nan: "No Response"})
df = df.applymap(str)
df = df[df[predictedVariable] != "No Response"]


MasterValentine_UpdatedCODIS_Feb12_2021.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,5,9,301) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#Filter out Idaho Data

In [ ]:
print(df['Site'].value_counts())
idaho = df[df['Site'] == '6'] #filtered dataframe without idaho data
df = df[df['Site'] != '6'] #filtered dataframe without idaho data
print(len(df))


1              2835
6              1017
4               742
5               675
2               247
3               125
No Response       2
Name: Site, dtype: int64
4626


#Filter out Genders

In [ ]:
#Code to filter out all other genders
df = df[df['Gender'] == '1'] #dataframe containing information from only female respondents

#Strangled

In [ ]:
eligible = df[df['CODISNDISeligibleProfile'] != 'No Response']
eligible = eligible[eligible['SuspectactionsSTRANGLEDCHOKED'] != 'No Response']

# eligible = eligible[eligible['SuspectactionsSTRANGLEDCHOKED'] != '2'] #Trying it without 'unknown'

newContingency= pd.crosstab(eligible['SuspectactionsSTRANGLEDCHOKED'], eligible["CODISNDISeligibleProfile"])
c, p, dof, expected = chi2_contingency(newContingency) 
print("The p-value obtained was " + str(p))
print(str(p) + " p-value")
print(str(dof) + " dof")
print(str(c) + " chi-square value")
print(newContingency)

The p-value obtained was 1.6437552593339485e-15
1.6437552593339485e-15 p-value
2 dof
68.08358595675413 chi-square value
CODISNDISeligibleProfile          0    1
SuspectactionsSTRANGLEDCHOKED           
0                              1794  987
1                               378  286
2                               708  224


#SuspectactionsGRABBEDHELD

In [ ]:
eligible = df[df['CODISNDISeligibleProfile'] != 'No Response']
eligible = eligible[eligible['SuspectactionsGRABBEDHELD'] != 'No Response']

# eligible = eligible[eligible['SuspectactionsGRABBEDHELD'] != '2'] #Trying it without 'unknown'

newContingency= pd.crosstab(eligible['SuspectactionsGRABBEDHELD'], eligible["CODISNDISeligibleProfile"])
c, p, dof, expected = chi2_contingency(newContingency) 
print("The p-value obtained was " + str(p))
print(str(p) + " p-value")
print(str(dof) + " dof")
print(str(c) + " chi-square value")
print(newContingency)

The p-value obtained was 8.81759267696509e-14
8.81759267696509e-14 p-value
2 dof
60.11888481654441 chi-square value
CODISNDISeligibleProfile      0     1
SuspectactionsGRABBEDHELD            
0                           532   234
1                          1635  1022
2                           714   241


#SuspectactionsPHYSICALBLOWS

In [ ]:
eligible = df[df['CODISNDISeligibleProfile'] != 'No Response']
eligible = eligible[eligible['SuspectactionsPHYSICALBLOWS'] != 'No Response']

# eligible = eligible[eligible['SuspectactionsPHYSICALBLOWS'] != '2']


newContingency= pd.crosstab(eligible['SuspectactionsPHYSICALBLOWS'], eligible["CODISNDISeligibleProfile"])
c, p, dof, expected = chi2_contingency(newContingency) 
print("The p-value obtained was " + str(p))
print(str(p) + " p-value")
print(str(dof) + " dof")
print(str(c) + " chi-square value")
print(newContingency)

The p-value obtained was 3.565521079248744e-11
3.565521079248744e-11 p-value
2 dof
48.114251628924364 chi-square value
CODISNDISeligibleProfile        0     1
SuspectactionsPHYSICALBLOWS            
0                            1726  1033
1                             439   223
2                             716   240


#Physicalinjury

In [ ]:
eligible = df[df['CODISNDISeligibleProfile'] != 'No Response']
eligible = eligible[eligible['Physicalinjury'] != 'No Response']

# eligible = eligible[eligible['Physicalinjury'] != '2']


newContingency= pd.crosstab(eligible['Physicalinjury'], eligible["CODISNDISeligibleProfile"])
c, p, dof, expected = chi2_contingency(newContingency) 
print(str(p) + " p-value")
print(str(dof) + " dof")
print(str(c) + " chi-square value")
print(newContingency)

0.01142890351128756 p-value
2 dof
8.943219473096121 chi-square value
CODISNDISeligibleProfile     0     1
Physicalinjury                      
0                          842   379
1                         2036  1118
2                           12     3


# Calculate Average Number of Assultive Acts

In [ ]:
eligible = df[df['CODISNDISeligibleProfile'] != 'No Response']
print(len(eligible))

currentVariable = 'SuspectactionsSTRANGLEDCHOKED'
strangled = eligible[eligible[currentVariable] != 'No Response']
strangled['CODISNDISeligibleProfile'] = np.where((strangled['CODISNDISeligibleProfile'] == '0'), 'Not Eligible', strangled['CODISNDISeligibleProfile'])
strangled['CODISNDISeligibleProfile'] = np.where((strangled['CODISNDISeligibleProfile'] == '1'), 'Eligible', strangled['CODISNDISeligibleProfile'])
strangled[currentVariable] = np.where((strangled[currentVariable] == '0'), 'No', strangled[currentVariable])
strangled[currentVariable] = np.where((strangled[currentVariable] == '1'), 'Yes', strangled[currentVariable])
strangled[currentVariable] = np.where((strangled[currentVariable] == '2'), 'Unknown', strangled[currentVariable])

#Remove "No Response" from both PostAssaultBathed and timebetweenassaultandexam
strangled = strangled[strangled['SuspectactionsSTRANGLEDCHOKED'] != 'No Response']
strangled = strangled[strangled['NumberAssaultiveActs'] != 'No Response']

strangled = strangled.apply(pd.to_numeric, errors='ignore') # convert all columns of DataFrame
strangled['CODISNDISeligibleProfile'] = np.where((strangled['CODISNDISeligibleProfile'] == '0'), 'Not Eligible', strangled['CODISNDISeligibleProfile'])
strangled['CODISNDISeligibleProfile'] = np.where((strangled['CODISNDISeligibleProfile'] == '1'), 'Eligible', strangled['CODISNDISeligibleProfile'])
strangled[currentVariable] = np.where((strangled[currentVariable] == '0'), 'No', strangled[currentVariable])
strangled[currentVariable] = np.where((strangled[currentVariable] == '1'), 'Yes', strangled[currentVariable])
strangled[currentVariable] = np.where((strangled[currentVariable] == '2'), 'Unknown', strangled[currentVariable])

print("Average number of assultive acts: " + str(strangled['NumberAssaultiveActs'].mean()))

strangledYes = strangled[strangled['SuspectactionsSTRANGLEDCHOKED'] == 'Yes']
print("Number strangled: " + str(len(strangledYes)))
print("Average number of assultive acts with strangulation: " + str(strangledYes['NumberAssaultiveActs'].mean()))


strangledNo = strangled[strangled['SuspectactionsSTRANGLEDCHOKED'] == 'No']
print("Number not strangled: " + str(len(strangledNo)))
print("Average number of assultive acts without strangulation: " + str(strangledNo['NumberAssaultiveActs'].mean()))

strangledUnknown = strangled[strangled['SuspectactionsSTRANGLEDCHOKED'] == 'Unknown']
print("Number strangulation unknown: " + str(len(strangledUnknown)))
print("Average number of assultive acts strangulation unknown: " + str(strangledUnknown['NumberAssaultiveActs'].mean()))


4404


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

Average number of assultive acts: 2.4558823529411766
Number strangled: 661
Average number of assultive acts with strangulation: 2.2072617246596065
Number not strangled: 2774
Average number of assultive acts without strangulation: 1.9780100937274694
Number strangulation unknown: 917
Average number of assultive acts strangulation unknown: 4.080697928026172
